### Runs an extended Kalman filter using IHME SEIIR predictions along with measurement data of confirmed Covid-19 case counts (from New York Times data) and Facebook symptom data (loss of smell/taste, from Covid-19 Symptom Challenge) to generate updated 7-day predictions of case counts for counties in New York State.

Developed by the University of Washington team of Les Atlas, Abraham Flaxman and Michael Rhoads.

S - Susceptible
E - Exposed
I1 - Presymptomatic
I2 - Symptomatic
R - Recovered

In [1]:
#%load_ext autoreload
#%autoreload

In [2]:
#%reset

In [3]:
import math
import datetime

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

import data_sets
import seiir_compartmental

In [4]:
# functions to support the Kalman filtering
def get_predicts_prior(day, seiir):
    x_hat = np.array([[seiir['S'].loc[day]],
                      [seiir['E'].loc[day]],
                      [seiir['I1'].loc[day]],
                      [seiir['I2'].loc[day]],
                      [seiir['R'].loc[day]]])

    beta_k = seiir['beta_pred'].loc[day]

    return x_hat, beta_k


def step_seiir(x_hat, constants, beta_k, days=7):
    s_dict = {'S': x_hat[0, 0],
              'E': x_hat[1, 0],
              'I1': x_hat[2, 0],
              'I2': x_hat[3, 0],
              'R': x_hat[4, 0]}

    s = pd.Series(s_dict)

    for i in range(days):
        infectious = s.loc['I1'] + s.loc['I2']
        s = seiir_compartmental.compartmental_covid_step(s, s.sum(),
                                                         infectious,
                                                         constants['alpha'],
                                                         beta_k,
                                                         constants['gamma1'],
                                                         constants['gamma2'],
                                                         constants['sigma'],
                                                         constants['theta'])
    x_hat_future_prior = np.array([[s.loc['S']],
                                   [s.loc['E']],
                                   [s.loc['I1']],
                                   [s.loc['I2']],
                                   [s.loc['R']]])

    return x_hat_future_prior


def predict_step(x_hat_k1_prior, P, Q, beta_k, constants):
    S = x_hat_k1_prior[0, 0]
    E = x_hat_k1_prior[1, 0]
    I1 = x_hat_k1_prior[2, 0]
    I2 = x_hat_k1_prior[3, 0]
    R = x_hat_k1_prior[4, 0]
    N = S + E + I1 + I2 + R
    alpha = constants['alpha']
    sigma = constants['sigma']
    gamma1 = constants['gamma1']
    gamma2 = constants['gamma2']

    part_f_S = np.array([[-beta_k * math.pow(I1 + I2, alpha) / N],
                         [beta_k * math.pow(I1 + I2, alpha) / N],
                         [0],
                         [0],
                         [0]])

    part_f_E = np.array([[0],
                         [-sigma],
                         [sigma],
                         [0],
                         [0]])

    part_f_I1 = np.array([[-alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [-gamma1],
                          [gamma1],
                          [0]])

    part_f_I2 = np.array([[-alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [0],
                          [-gamma2],
                          [gamma2]])

    part_f_R = np.array([[0],
                         [0],
                         [0],
                         [0],
                         [0]])

    # 5x5
    f_jacob = np.concatenate([part_f_S, part_f_E, part_f_I1, part_f_I2,
                              part_f_R], axis=1)

    # 5x5
    # P_k1_prior = f_jacob * P * f_jacob^T + Q
    P_k1_prior = np.matmul(np.matmul(f_jacob, P), np.transpose(f_jacob)) + Q
    return P_k1_prior


def update_step(x_hat, x_hat_k1, P_k1, Rn, rho1, rho2, z_k):
    # 5x5
    ep = 10**-8
    
    H = np.array([[ep, 0, 0, 0, 0],
                  [0, ep, 0, 0, 0],
                  [0, 0, ep, rho1, 0],
                  [0, 0, 0, rho2, 0],
                  [0, 0, 0, 0, ep]])
    
    
    # Si = H * P_k1 * H^T + Rn
    Si = np.matmul(np.matmul(H, P_k1), np.transpose(H)) + Rn

    # K_new = P_k1 * H^T * Si^(-1)
    K_new = np.matmul(np.matmul(P_k1, np.transpose(H)), np.linalg.inv(Si))
    y_new = np.matmul(H, x_hat)

    # 5x1
    diff = z_k - y_new

    x_hat_k1_post = x_hat_k1 + np.matmul(K_new, diff)

    P_k1_post = P_k1 - np.matmul(np.matmul(K_new, Si), np.transpose(K_new))


    return x_hat_k1_post, P_k1_post, diff


def get_data_sets(state, fips=None):
    # the post hoc seiir model predictions provided by Prof Flaxman without
    # any kalman filtering:
    seiir = pd.read_csv(r'data/seiir_compartments_post-hoc_ny_state.csv', header=0,
                        index_col='date', parse_dates=True)
    

    fb_data = data_sets.create_symptom_df()
    fb_data_val = data_sets.create_symptom_df(valid=True)

    if fips is None:
        case_data = data_sets.create_case_df_state()
        case_data_geo = case_data.loc[state]['case_rate'].copy()
        fb_data_geo = fb_data.loc[state].groupby('date').sum().copy()
        fb_data_val_geo = fb_data_val.loc[state].groupby('date').sum().copy()

    else:
        case_data = data_sets.create_case_df_county()
        case_data_geo = case_data.loc[fips]['case_rate'].copy()
        
        if fips == 'New York City':
            nyc_fips = ['36005', '36061', '36047', '36085']
            fb_data_geo = fb_data.loc[(slice(None), '36081'), :].copy()
            fb_data_geo = fb_data_geo.mean(level='date')
            fb_data_val_geo = fb_data_val.loc[(slice(None), '36081'), :].copy()
            fb_data_val_geo = fb_data_val.mean(level='date')
            for borough in nyc_fips:
                fb_data_geo += fb_data.loc[(slice(None), borough), :].copy().mean(level='date')
                fb_data_val_geo += fb_data_val.loc[(slice(None), borough), :].copy().mean(level='date')
        else:
            fb_data_geo = fb_data.loc[(slice(None), fips), :].copy()
            fb_data_val_geo = fb_data_val.loc[(slice(None), fips), :].copy()
            # collapse down to a single index column (date)
            fb_data_geo = fb_data_geo.mean(level='date')
            fb_data_val_geo = fb_data_val_geo.mean(level='date')


    return seiir, fb_data_geo, fb_data_val_geo, case_data_geo


def calc_fb_ma7(fb_data):
    """
    Returns a Pandas series
    """
    # the fb_data is a DataFrame while the case_data is a Series
    fb_ma7 = fb_data.rolling(window=7).mean()
    fb_ma7 = fb_ma7.iloc[6:, :]
    prop_ma7 = fb_ma7['num_stl'].div(fb_ma7['n'])

    return prop_ma7, fb_ma7['n'].copy(), fb_ma7['num_stl'].copy()

In [8]:
# bring in new data
full_data = pd.read_csv(r'data/from_challenge/overall-county.csv', header=0, dtype={'fips': 'str', 'pct_hh_cli': 'float64'},
                        parse_dates=[0])
full_data.set_index(['fips', 'date'], inplace=True)
full_data.sort_index(inplace=True)


In [10]:
data_sets.get_fips('NY', 'Albany')

'36001'

In [13]:
full_data

state_code   gender age_bucket   n    weight_sums  pct_cli  \
fips  date                                                                    
01001 2020-04-07         al  overall    overall  58   34336.880139   0.0000   
      2020-04-08         al  overall    overall  58   37450.490430   0.0000   
      2020-04-09         al  overall    overall  54   44925.319666   0.0000   
01003 2020-04-06         al   female    overall  56   79915.492427   0.0000   
      2020-04-06         al  overall    overall  72  128811.417352   0.0000   
...                     ...      ...        ...  ..            ...      ...   
56025 2020-04-08         wy  overall    overall  82   44796.741492   1.2500   
      2020-04-09         wy   female    overall  61   31802.546566   4.9180   
      2020-04-09         wy  overall    overall  82   58574.732197   3.6585   
      2020-04-14         wy  overall    overall  61   84947.819894   0.0000   
      2020-05-22         wy  overall    overall  51  100471.423495   0.0000   

                  pct_ili  pct_cli_anosmia_ageusia  pct_hh_cli  pct_cmnty_cli  \
fips  date                                                                      
01001 2020-04-07   0.0000                   1.7241      0.0000            NaN   
      2020-04-08   0.0000                   0.0000      0.0000            NaN   
      2020-04-09   0.0000                   1.8868      1.8519            NaN   
01003 2020-04-06   0.0000                   3.5714      1.7857            NaN   
      2020-04-06   0.0000                   4.2857      1.3889            NaN   
...                   ...                      ...         ...            ...   
56025 2020-04-08   1.2500                   3.7500      2.4691            NaN   
      2020-04-09   4.9180                   6.5574      6.5574            NaN   
      2020-04-09   3.6585                   6.0976      4.8780            NaN   
      2020-04-14   0.0000                   3.2787      0.0000            NaN   
      2020-05-22   0.0000                   1.9608      0.0000           12.0   

                  ...  pct_diabetes_weighted  pct_cancer_weighted  \
fips  date        ...                                               
01001 2020-04-07  ...                22.5362               8.8237   
      2020-04-08  ...                11.5485               5.6571   
      2020-04-09  ...                10.5565               8.6076   
01003 2020-04-06  ...                 6.2391               3.0230   
      2020-04-06  ...                24.5611               1.8755   
...               ...                    ...                  ...   
56025 2020-04-08  ...                 5.4801               0.7561   
      2020-04-09  ...                 4.1394               4.0399   
      2020-04-09  ...                14.3441               2.1934   
      2020-04-14  ...                 9.0400               2.1684   
      2020-05-22  ...                 6.2641               5.9735   

                  pct_heart_disease_weighted  \
fips  date                                     
01001 2020-04-07                      8.6917   
      2020-04-08                      4.0042   
      2020-04-09                      9.7792   
01003 2020-04-06                      8.1429   
      2020-04-06                      5.0519   
...                                      ...   
56025 2020-04-08                      4.7494   
      2020-04-09                      9.5186   
      2020-04-09                      5.1680   
      2020-04-14                      5.8564   
      2020-05-22                     11.7572   

                  pct_high_blood_pressure_weighted  pct_asthma_weighted  \
fips  date                                                                
01001 2020-04-07                           33.3613               8.5199   
      2020-04-08                           27.3263              10.3085   
      2020-04-09                           33.8916              10.0756   
01003 2020-04-06                     

In [5]:
# set constants
#K0 = datetime.date(2020, 4, 12)
K0 = datetime.date(2020, 4, 18)

the_state = 'NY'
#the_county = data_sets.get_fips(the_state, 'Erie')
the_county = 'New York City'
constants = {
    'alpha': 0.948786,
    'gamma1': 0.500000,
    'gamma2': 0.662215,
    'sigma': 0.266635,
    'theta': 6.000000
    }

# set initial values for Kalman filter parameters
P_mult = 1
Q_mult = 1

# Rn is the R noise covariance matrix; it remains constant thru the stepping of the
# Kalman filter
Rn_mult = 5*10**-8

Rn_22 = 10000
Rn_32 = 1000

Rn_23 = 1000
Rn_33 = 100

Rn = Rn_mult * np.array([[0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0],
                         [0, 0, Rn_22, Rn_23, 0],
                         [0, 0, Rn_32, Rn_33, 0],
                         [0, 0, 0, 0, 0]])

Q = Q_mult * np.eye(5)
P = P_mult * np.eye(5)

# generate data
seiir, fb_data, fb_data_val, case_data = get_data_sets(
    data_sets.STATES[the_state], fips=the_county)


if the_county == 'New York City':
    county_pop = 0
    for each in ['36081', '36005', '36061', '36047', '36085']:
        this_count, state_pop = data_sets.get_pops(each)
        county_pop += this_count
else:
    county_pop, state_pop = data_sets.get_pops(the_county)

b = county_pop / state_pop

# calculate moving averages on the fb and case data
case_ma7 = case_data.rolling(window=7).mean()
case_ma7_all = case_ma7.iloc[6:]
prop_ma7, n_ma7, num_stl_ma7 = calc_fb_ma7(fb_data)

prop_ma7_valid, n_ma7_valid, num_stl_ma7_valid = calc_fb_ma7(fb_data_val)

# get starting compartment values for the state level
x_hat_state_k0, beta_k0 = get_predicts_prior(K0, seiir)

# convert to the county level
x_hat_k0 = b * x_hat_state_k0

I2_county = x_hat_k0[3, 0]


rho1 = 0.0001
rho2 = case_ma7_all.loc['2020-04-12'] / I2_county


# create empty dictionaries to hold the estimated values
prop_est = {}
case_est = {}
seiir_pred = {}

diff_rat = {}



# Original data run ----------------
start = K0
d = start
#while d <= datetime.date(2020, 9, 30):
while d <= datetime.date(2020, 10, 23):    

# each cycle of the while loop executes a step

    # get state level compartments
    x_hat_state_k, beta_k = get_predicts_prior(d, seiir)

    # step the state level compartments 7 days forward
    x_hat_state_k1 = step_seiir(x_hat_state_k, constants, beta_k)
    
    # convert the state level compartments to county level values
    x_hat_k = b * x_hat_state_k
    x_hat_k1 = b * x_hat_state_k1


    # get measurements for current day
    
    z_k = np.array([[0],
                    [0],
                    [prop_ma7.loc[d - datetime.timedelta(days=6)]],
                    [case_ma7_all.loc[d]],
                    [0]])

    # predict step using the stepped fwd SEIIR compartment values 
    P = predict_step(x_hat_k1, P, Q, beta_k, constants)
    print('step -----------')
    print('P:')
    print(P)

    # update step
    x_hat_post, P_post, the_diff = update_step(x_hat_k, x_hat_k1, P, Rn,
                                     rho1, rho2, z_k)
    print('P_post:')
    print(P_post)
    

    # store estimated values for proportion and case rate
    indexDate = d + datetime.timedelta(days=7)
    prop_est[indexDate] = rho1 * x_hat_post[3, 0]
    case_est[indexDate] = rho2 * x_hat_post[3, 0]
    seiir_pred[indexDate] = x_hat_k1[3, 0]
    
    diff_rat[indexDate] = the_diff[2, 0] / the_diff[3, 0]

    # update the P and d
    P = P_post
    d += datetime.timedelta(days=1)
    
# create pandas series of the estimated case rate
predicted_case = pd.Series(case_est)
predicted_seiir_prior = pd.Series(seiir_pred)

step -----------
P:
[[ 1.10348197 -0.10348197  0.11373077  0.03689768 -0.15062845]
 [-0.10348197  1.1745762  -0.184825   -0.03689768  0.15062845]
 [ 0.11373077 -0.184825    1.32109422 -0.25        0.        ]
 [ 0.03689768 -0.03689768 -0.25        1.68852871 -0.43852871]
 [-0.15062845  0.15062845  0.         -0.43852871  1.43852871]]
P_post:
[[ 4.44089210e-16  1.80411242e-16  6.52256027e-16 -4.25354196e-15
   1.13797860e-15]
 [ 1.80411242e-16  0.00000000e+00  5.55111512e-17  0.00000000e+00
  -2.77555756e-17]
 [ 6.66133815e-16  0.00000000e+00  1.24291957e+00  8.88454277e-09
   1.10756825e-17]
 [-4.26741975e-15 -6.93889390e-18  8.88454277e-09 -6.66133815e-16
   8.88178420e-16]
 [ 1.16573418e-15 -2.77555756e-17  9.48006304e-18  8.88178420e-16
  -8.88178420e-16]]
step -----------
P:
[[ 1.06396006e+00 -6.39600636e-02  1.40976252e-01 -1.40976251e-01
  -1.33464876e-09]
 [-6.39600636e-02  1.06396006e+00 -1.40976252e-01  1.40976251e-01
   1.33464876e-09]
 [ 1.40976252e-01 -1.40976252e-01  1.310

step -----------
P:
[[ 1.06151995e+00 -6.15199461e-02  1.36563320e-01 -1.36563319e-01
  -1.29287089e-09]
 [-6.15199461e-02  1.06151995e+00 -1.36563320e-01  1.36563319e-01
   1.29287089e-09]
 [ 1.36563320e-01 -1.36563320e-01  1.30314624e+00 -3.03146239e-01
  -2.86994300e-09]
 [-1.36563319e-01  1.36563319e-01 -3.03146239e-01  1.30314624e+00
   2.86994290e-09]
 [-1.29287089e-09  1.29287089e-09 -2.86994299e-09  2.86994290e-09
   1.00000000e+00]]
P_post:
[[ 6.66133815e-16  1.73472348e-16  8.32667268e-17 -2.77555756e-16
  -3.51551760e-24]
 [ 1.73472348e-16  2.22044605e-16  2.49800181e-16 -1.16573418e-15
  -1.03397577e-24]
 [ 1.38777878e-16  2.22044605e-16  1.21255727e+00  8.66750971e-09
  -4.13590306e-25]
 [-3.05311332e-16 -1.16573418e-15  8.66750971e-09  2.22044605e-16
   1.24077092e-24]
 [-3.72231276e-24 -1.24077092e-24  4.42709132e-18  1.65436123e-24
   1.11022302e-16]]
step -----------
P:
[[ 1.06172781e+00 -6.17278110e-02  1.36792275e-01 -1.36792274e-01
  -1.29503846e-09]
 [-6.17278110e-

KeyboardInterrupt: 

In [ ]:
diff_timeser = pd.Series(diff_rat)

In [ ]:
plt.plot(diff_timeser.index, diff_timeser)
plt.title('NYC')

In [ ]:
n = len(prop_ma7.values)
f_t = np.fft.fft(prop_ma7.values, n)

mag = np.sqrt(f_t * np.conj(f_t))
freq = (1/n)* np.arange(n)
L = np.arange(1, np.floor(n/2), dtype='int64')
plt.plot(freq[L], mag[L])

In [ ]:
prop_ma7.values

In [ ]:
# error computations

In [ ]:
# compute squared error for the overlapping time period -------------------------

left = predicted_seiir_prior.index[0]
right = d

seiir_err = case_ma7_all.loc[left:right] - predicted_seiir_prior[left:right]
seiir_sum_sq_err = (seiir_err**2).sum()
seiir_mse = seiir_sum_sq_err / seiir_err.count()

kalman_err = predicted_case.loc[left:right] - case_ma7_all.loc[left:right]
kalman_sum_sq_err = (kalman_err**2).sum()
kalman_mse = kalman_sum_sq_err / kalman_err.count()


print('SSE between seiir forecast and case rate:', seiir_sum_sq_err)
print('MSE between seiir forecast and measured case rate:', seiir_mse)
print()
print('SSE between kalman forecast and case rate:', kalman_sum_sq_err)
print('MSE between kalman forecast and measured case rate:', kalman_mse)

In [ ]:
# MASE calculation -----------------

left = predicted_case.index[0]
right = predicted_case.index[-1]

# take error between prediction and actual
e = (case_ma7_all.loc[left:right] - predicted_case.loc[left:right]).abs()

case_ma7_diff = case_ma7_all.diff(7)
denom = (case_ma7_diff.loc[left:right]).abs()

term = e.div(denom)
term.replace([np.inf, -np.inf], np.nan, inplace=True)
term.dropna(inplace=True)
mase = term.sum() / len(term)
print('mase:', mase)

# All plotting code below this

In [ ]:
# Plotting constants and variables ----------------
matplotlib.rcParams.update({'font.size': 22})
plt.style.use('seaborn-whitegrid')
purple = '#33016F'
gold = '#9E7A27'
gray = '#797979'
width = 4
%matplotlib qt

tick_end = predicted_seiir_prior.index[-1]
special_start = datetime.date(2020, 8, 16)

week_interval = pd.date_range(start=special_start, end=tick_end, freq='W')
week_interval = [x.to_pydatetime().date() for x in week_interval]

In [ ]:
# Single plot with all lines
# Single plot----------

fig3, ax31 = plt.subplots(1)
plt.sca(ax31)
plt.plot(case_ma7_all.loc[special_start:d].index, case_ma7_all.loc[special_start:d], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.loc[special_start:tick_end].index, predicted_seiir_prior.loc[special_start:tick_end],
         label='IHME 7-day Forecast', c=gray, linewidth=width)
plt.plot(predicted_case.loc[special_start:tick_end].index, predicted_case.loc[special_start:tick_end], label='Our 7-Day Forecast',
         c=purple, linewidth=width)
plt.plot(case_ma7_all.loc[d:tick_end].index, case_ma7_all.loc[d:tick_end], c='darkorange',
         linewidth=width)
plt.ylim(-50, 1300)
#plt.ylim(-2, 72)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')
# plt.xlabel('Date')
plt.xlim(special_start, datetime.date(2020, 11, 3))
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')

ax32 = ax31.twinx()
plt.sca(ax32)
plt.plot(num_stl_ma7.loc[special_start:d].index, num_stl_ma7.loc[special_start:d], c='red', label='Facebook Positive Symptoms, Smell/Taste Loss',
         linewidth=width)
plt.plot(num_stl_ma7.loc[d:tick_end].index, num_stl_ma7.loc[d:tick_end], c='maroon',
         linewidth=width)
#plt.plot(prop_ma7.index, num_stl_ma7, c='red', label='Facebook Positive Symptoms',
#         linewidth=width)
#plt.ylim(-.065, 2.5)
plt.ylim(-.4, 11)
plt.grid(axis='y', linestyle=':')
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')

plt.ylabel('Number of Positive Symptom Response per Day')
plt.legend(loc='upper right')

plt.show()

In [ ]:
xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 10, 13)

leftylim_low = -700
leftylim_high = 20000

rightylim_low = -3
rightylim_high = 85

xtick_size = 14
xlabel_size = 14

In [ ]:
# plot findings -- multiple plots

# Plotting constants and variables ----------------

plt.style.use('seaborn-whitegrid')
matplotlib.rcParams.update({'font.size': 18})
purple = '#33016F'
gold = '#9E7A27'
gray = '#797979'
width = 4
%matplotlib qt

tick_end = predicted_seiir_prior.index[-1]

week_interval = pd.date_range(start=start, end=tick_end, freq='W')
week_interval = [x.to_pydatetime().date() for x in week_interval]



In [ ]:
# multiple plots -----------
fig1, ax11 = plt.subplots(1)
plt.sca(ax11)
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)


fig2, ax21 = plt.subplots(1)
plt.sca(ax21)
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)




fig3, ax31 = plt.subplots(1)
plt.sca(ax31)
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)

ax32 = ax31.twinx()
plt.sca(ax32)
plt.plot(num_stl_ma7.loc[start:d].index, num_stl_ma7.loc[start:d], c='red', label='FB Positive Symptoms, Smell/Taste Loss',
         linewidth=width)
plt.grid(axis='y', linestyle=':')

plt.ylabel('Number of Positive Symptom Response per Day')
plt.legend(loc='upper right')
plt.ylim(rightylim_low, rightylim_high)


plt.sca(ax31)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')

plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)






In [ ]:
fig4, ax41 = plt.subplots(1)
plt.sca(ax41)
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)
#naieve estimate
naive_start = start + datetime.timedelta(days=7)
naive_d = d + datetime.timedelta(days=7)
plt.plot(case_ma7_all.loc[naive_start:naive_d].index, case_ma7_all.loc[start:d], label='Naive guess', c='orange')

ax42 = ax41.twinx()
plt.sca(ax42)
plt.plot(num_stl_ma7.loc[start:d].index, num_stl_ma7.loc[start:d], c='red', label='FB Positive Symptoms, Smell/Taste Loss',
         linewidth=width)
plt.grid(axis='y', linestyle=':')

plt.ylabel('Number of Positive Symptom Response per Day')
plt.legend(loc='upper right')
plt.ylim(rightylim_low, rightylim_high)


plt.sca(ax41)
plt.plot(predicted_case.loc[start:].index, predicted_case.loc[start:], label='Our 7-Day Forecast',
         c=purple, linewidth=width)

plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')

plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)

In [ ]:
fig5, ax51 = plt.subplots(1)
plt.sca(ax51)
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Confirmed Case Count', c=gold,
         linewidth=width)

plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)

ax52 = ax51.twinx()
plt.sca(ax52)
plt.plot(num_stl_ma7.loc[start:d].index, num_stl_ma7.loc[start:d], c='red', label='FB Positive Symptoms, Smell/Taste Loss',
         linewidth=width)
plt.grid(axis='y', linestyle=':')

plt.ylabel('Number of Positive Symptom Response per Day')
plt.legend(loc='upper right')
plt.ylim(rightylim_low, rightylim_high)


plt.sca(ax51)
plt.plot(predicted_case.loc[start:].index, predicted_case.loc[start:], label='Our 7-Day Forecast',
         c=purple, linewidth=width)
plt.plot(case_ma7_all.loc[d:tick_end].index, case_ma7_all.loc[d:tick_end], label='Confirmed Case Count', c='yellow',
         linewidth=width)

plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')

plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)


In [ ]:
plt.show()

In [ ]:
## special end date

# plot findings -- multiple plots

# Plotting constants and variables ----------------
matplotlib.rcParams.update({'font.size': 18})
plt.style.use('seaborn-whitegrid')
purple = '#33016F'
gold = '#9E7A27'
gray = '#797979'
width = 4
%matplotlib qt

tick_end = predicted_seiir_prior.index[-1]

week_interval = pd.date_range(start=start, end=tick_end, freq='W')
week_interval = [x.to_pydatetime().date() for x in week_interval]


fig1, ax11 = plt.subplots(1)
plt.sca(ax11)
plt.plot(case_ma7_all.loc[start:tick_end].index, case_ma7_all.loc[start:tick_end], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)


fig2, ax21 = plt.subplots(1)
plt.sca(ax21)
plt.plot(case_ma7_all.loc[start:tick_end].index, case_ma7_all.loc[start:tick_end], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)




fig3, ax31 = plt.subplots(1)
plt.sca(ax31)
plt.plot(case_ma7_all.loc[start:tick_end].index, case_ma7_all.loc[start:tick_end], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)

ax32 = ax31.twinx()
plt.sca(ax32)
plt.plot(num_stl_ma7.loc[start:tick_end].index, num_stl_ma7.loc[start:tick_end], c='red', label='FB Positive Symptoms, Smell/Taste Loss',
         linewidth=width)
plt.grid(axis='y', linestyle=':')

plt.ylabel('Number of Positive Symptom Response per Day')
plt.legend(loc='upper right')
plt.ylim(rightylim_low, rightylim_high)


plt.sca(ax31)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')

plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)



fig4, ax41 = plt.subplots(1)
plt.sca(ax41)
plt.plot(case_ma7_all.loc[start:tick_end].index, case_ma7_all.loc[start:tick_end], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)

ax42 = ax41.twinx()
plt.sca(ax42)
plt.plot(num_stl_ma7.loc[start:tick_end].index, num_stl_ma7.loc[start:tick_end], c='red', label='FB Positive Symptoms, Smell/Taste Loss',
         linewidth=width)
plt.grid(axis='y', linestyle=':')

plt.ylabel('Number of Positive Symptom Response per Day')
plt.legend(loc='upper right')
plt.ylim(rightylim_low, rightylim_high)


plt.sca(ax41)
plt.plot(predicted_case.loc[start:].index, predicted_case.loc[start:], label='Our 7-Day Forecast',
         c=purple, linewidth=width)

plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')

plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)


In [ ]:
# SHORTENED Erie plot settings
xlim_left = datetime.date(2020, 5, 24)
xlim_right = datetime.date(2020, 10, 13)

leftylim_low = -72
leftylim_high = 900

rightylim_low = -1
rightylim_high = 12.5

xtick_size = 14
xlabel_size = 14

In [ ]:
# SHORTENED Albany plot settings
xlim_left = datetime.date(2020, 5, 24)
xlim_right = datetime.date(2020, 10, 13)

leftylim_low = -20
leftylim_high = 275

rightylim_low = -.225
rightylim_high = 3.1

xtick_size = 14
xlabel_size = 14

In [ ]:
# SHORTENED Westchester plot settings
xlim_left = datetime.date(2020, 5, 24)
xlim_right = datetime.date(2020, 10, 13)

leftylim_low = -100
leftylim_high = 900

rightylim_low = -.75
rightylim_high = 7

xtick_size = 14
xlabel_size = 14

In [ ]:
# SHORTENED Nassau plot settings
xlim_left = datetime.date(2020, 5, 24)
xlim_right = datetime.date(2020, 10, 13)

leftylim_low = -110
leftylim_high = 1200

rightylim_low = -1.1
rightylim_high = 12

xtick_size = 14
xlabel_size = 14

In [ ]:
# SHORTENED NYC plot settings

xlim_left = datetime.date(2020, 5, 24)
xlim_right = datetime.date(2020, 10, 13)

leftylim_low = -295
leftylim_high = 8000

rightylim_low = -3
rightylim_high = 80

xtick_size = 14
xlabel_size = 14

In [ ]:
# NYC plot settings

xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 11, 3)

leftylim_low = -700
leftylim_high = 12000

rightylim_low = -5
rightylim_high = 85

xtick_size = 14
xlabel_size = 14

In [ ]:
# Nassau plot settings
xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 11, 3)

leftylim_low = -40
leftylim_high = 1650

rightylim_low = -.3
rightylim_high = 12

xtick_size = 14
xlabel_size = 14

In [ ]:
# Westchester plot settings
xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 11, 3)

leftylim_low = -28
leftylim_high = 1000

rightylim_low = -.2
rightylim_high = 7

xtick_size = 14
xlabel_size = 14

In [ ]:
# Albany plot settings
xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 11, 3)

leftylim_low = -3
leftylim_high = 75

rightylim_low = -.125
rightylim_high = 3.1

xtick_size = 14
xlabel_size = 14

In [ ]:
# Erie plot settings
xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 10, 13)

leftylim_low = -10
leftylim_high = 250

rightylim_low = -.5
rightylim_high = 12.5

xtick_size = 14
xlabel_size = 14